In [44]:
#https://github.com/gregversteeg/corex_topic

In [1]:
import nltk
import pandas as pd

In [4]:
from corextopic import corextopic as ct

# CorEx on Positive Articles

In [11]:
import pandas as pd

data = pd.read_csv("../data/indonesia/output/2018/01.csv",index_col=0)

data.reset_index(drop=True, inplace=True)

data.head()

,Conflict
0,False
1,False
2,False
3,False
4,True


In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(
    max_df=.5,
    min_df=0.95,
    max_features=None,
    ngram_range=(1, 2),
    norm=None,
    binary=True,
    use_idf=False,
    sublinear_tf=False,
    strip_accents = 'unicode',
    stop_words = 'english'
)

vectorizer = vectorizer.fit(wri['text_coref'])
tfidf = vectorizer.transform(wri['text_coref'])
vocab = vectorizer.get_feature_names()
print(len(vocab))
print(vocab)


121934
['00', '00 000', '00 23', '00 28', '00 52', '00 54', '00 80', '000', '000 000', '000 14', '000 1951', '000 2006', '000 2011', '000 2016', '000 2017', '000 25000', '000 40', '000 50', '000 acre', '000 acres', '000 additional', '000 affected', '000 annum', '000 ants', '000 army', '000 beneficiaries', '000 bhagirathi', '000 billion', '000 birds', '000 bulandshahr', '000 cheque', '000 chronically', '000 citizens', '000 commenced', '000 consumers', '000 continuous', '000 crore', '000 crores', '000 cubic', '000 cusecs', '000 dams', '000 die', '000 elephants', '000 families', '000 family', '000 farmers', '000 farming', '000 feet', '000 ha', '000 hectare', '000 hectares', '000 homeless', '000 homes', '000 hours', '000 households', '000 illegal', '000 ita', '000 karnataka', '000 kg', '000 kilometres', '000 kin', '000 km', '000 kms', '000 later', '000 litres', '000 locations', '000 marched', '000 megawatts', '000 metres', '000 metric', '000 million', '000 mld', '000 mm', '000 month', '000

In [8]:
TOPICS = 7
NBR_OF_WORDS = 3

In [9]:
anchors = []
model = ct.Corex(n_hidden=TOPICS, seed=42)
model = model.fit(
    tfidf,
    words=vocab
)

KeyboardInterrupt: 

## Keywords for Topic Anchors
['land', 'acre','hectares', 'acquisition', 'land acquisition', 'agricultural', 'acres', 'degradation','landslides','property','resettlement'],

['farmer', 'farming', 'agricultural', 'produce', 'crop', 'crops', 'agrarian', 'farms','farm','field','fields','soil','sugarcane','vegetables','farmers','agriculture','tractor','prices crops', 'debt','quota','food','fruits','livestock','cow','wheat','harvest','harvesting','horticulture','loan','loans','milk','paddy','rice','plant','plants','potatoes','potato'],

['mining', 'coal', 'miner', 'miners','sand mining', 'sand','bauxite','iron ore','limestone','manganese ore','granite'],

['forest','forests', 'forest department', 'reserve', 'forest officials','forestry'],

['animal','leopard','leopards', 'animals', 'wildlife', 'tiger', 'attacked', 'slaughter', 'lion','lions', 'threat', 'tigress', 'bear','birds','cat','cattle','crocodile','elephant','elephants','pangolin','pangolins','species'],

['drought', 'droughts','monsoon', 'rain','rains','rainfall','disaster'],

['water', 'irrigation', 'monsoon', 'rain', 'flood', 'floods', 'flooded', 'climate change','climate','dam','dams','drinking']



In [ ]:
# Anchors designed to nudge the model towards measuring specific genres

anchors = [
    ['land','resettlement','degradation','plot'],
    ['farm','Farmers','crop','agriculture','crops','agrarian','farmer','farmers''cows','tractor','acre','fields','livestock','harvest','harvesting','potato','sugarcane','paddy','rice','milk'],
    ['mining', 'coal', 'miner', 'miners','sand mining', 'sand','bauxite','iron ore','limestone','manganese ore','granite'],
    ['forest','deforestation','trees'],
    ['animal','attacked','leopard','leopards','tiger','tigress','crocodile'],
    ['drought','rain','climate change'],
    ['water','dams','irrigation','flood','drinking']   
   
]
anchors = [
    [a for a in topic if a in vocab]
    for topic in anchors
]

model = ct.Corex(n_hidden=TOPICS, seed=42)
model = model.fit(
    tfidf,
    words=vocab,
    anchors=anchors, # Pass the anchors in here
    anchor_strength=100 # Tell the model how much it should rely on the anchors
)

In [ ]:
for i, topic_ngrams in enumerate(model.get_topics(n_words=NBR_OF_WORDS)):
    topic_ngrams = [ngram[0] for ngram in topic_ngrams if ngram[1] > 0]
    print("Topic #{}: {}".format(i+1, ", ".join(topic_ngrams)))

In [ ]:
topic_df = pd.DataFrame(
    model.transform(tfidf), 
    columns=["topic_{}".format(i+1) for i in range(TOPICS)]
).astype(float)
topic_df.index = wri.index
wri = pd.concat([wri, topic_df], axis=1)

In [ ]:
for i in range(TOPICS):
    column='topic_{}'.format(i+1)
    print(wri[column].value_counts(normalize=True))

In [56]:
#Topic Flags
wri['topic']=wri['topic_1']+wri['topic_2']+wri['topic_3']+wri['topic_4']+wri['topic_5']+wri['topic_6']+wri['topic_7']
wri['topic'].value_counts(normalize=True)

2.0    0.335306
1.0    0.333333
3.0    0.147929
0.0    0.065089
4.0    0.061144
5.0    0.041420
6.0    0.015779
Name: topic, dtype: float64

In [59]:
MisTagged = wri[wri['topic']==0]
MisTagged.to_csv("MisTagged.csv")
print(MisTagged)
print(MisTagged.info())


        class                                               text  \
5    positive  After 20 years of providing Manitoba livestock...   
23   positive  The fire was restricted to one section of the ...   
71   positive  New Delhi, Oct 2 (IANS) Left parties on Tuesda...   
74   positive  Updated: Aug 29, 2019 21:05 IST\r\r\r\nIndia r...   
82   positive  This refers to ÃÂ¢ÃÂÃÂFarmers march in De...   
90   positive  Chinese troops intrude into Arunachal with roa...   
116  positive  farmersÃÂ¢ÃÂÃÂ protest\r\r\r\nNGO Nation\...   
129  positive  The 'Kisan Kranti Padyatra', which was started...   
141  positive  Accept the updated privacy & cookie policy\r\r...   
160  positive  Advertisement\r\r\n\r\r\nBellandur Lake Group,...   
238  positive  Nagpur, MAHARASHTRA ÃÂ¢ÃÂÃÂ Over one lakh...   
243  positive  A plea alleging that rainwater harvesting syst...   
257  positive  Prices for farmed vannamei shrimp should be fa...   
274  positive  Telangana IT and Industries Minis